In [4]:
import argparse
import datetime
import os
import sys
import timeit
import warnings

import SimpleITK as sitk
import copy
import itertools
import SimpleITK as sitk
import numpy as np

if sys.version_info[0:2] > (3, 9):
    raise Exception(
        "Python version above 3.9 may cause problems with SimpleITK. [BufferError: memoryview has 1 exported buffer]"
    )

try:
    import mialab.data.structure as structure
except ImportError:
    # Append the MIALab root directory to Python path
    sys.path.insert(0, os.path.join(os.path.dirname(sys.argv[0]), ".."))
    import mialab.data.structure as structure


In [5]:
def subtract_overlap(larger_structure, smaller_structure):
    # Convert images to arrays
    larger_array = sitk.GetArrayFromImage(larger_structure)
    smaller_array = sitk.GetArrayFromImage(smaller_structure)

    # Identify overlapping regions
    overlap_array = np.logical_and(larger_array > 0, smaller_array > 0)

    # Set overlapping parts to 0 in the larger structure
    larger_array = np.where(overlap_array, 0, larger_array)

    # Identify non-overlapping regions and keep them in the larger structure
    larger_array = np.where(np.logical_not(overlap_array), larger_array, 0)

    return sitk.GetImageFromArray(larger_array)


In [8]:

%cd C:\Users\FlipFlop\Documents\UniBE\Sem5_MedImLab\Repo\MIALab
#todo I had to add this
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# Replace these paths with the actual paths to your segmentation files
white_matter_path = "from_ubelix/white_matter/117122_SEG.mha"
gray_matter_path = "from_ubelix/grey_matter/117122_SEG.mha"
thalamus_path = "from_ubelix/thalamus_matter/117122_SEG.mha"
hippocampus_path = "from_ubelix/hippocampus_matter/117122_SEG.mha"
amygdala_path = "from_ubelix/amygdala_matter/117122_SEG.mha"

# Read segmentation images
white_matter = sitk.ReadImage(white_matter_path)
gray_matter = sitk.ReadImage(gray_matter_path)
thalamus = sitk.ReadImage(thalamus_path)
hippocampus = sitk.ReadImage(hippocampus_path)
amygdala = sitk.ReadImage(amygdala_path)

# Copy original images
white_matter_original = copy.deepcopy(white_matter)
gray_matter_original = copy.deepcopy(gray_matter)
thalamus_original = copy.deepcopy(thalamus)
hippocampus_original = copy.deepcopy(hippocampus)
amygdala_original = copy.deepcopy(amygdala)

# priority list 
priority_list = ['hypocampus', 'amygdala', 'thalamus', 'gray_matter', 'white_matter']

# substract overlap from larger structures
# white matter is substracted gray matter, thalamus, hippocampus, amygdala
# gray matter is substracted thalamus, hippocampus, amygdala
# thalamus is substracted hippocampus, amygdala
# amygdala is substracted hippocampus

# white matter
white_matter = subtract_overlap(white_matter, gray_matter)
white_matter = subtract_overlap(white_matter, thalamus)
white_matter = subtract_overlap(white_matter, hippocampus)
white_matter = subtract_overlap(white_matter, amygdala)

# gray matter
gray_matter = subtract_overlap(gray_matter, thalamus)
gray_matter = subtract_overlap(gray_matter, hippocampus)
gray_matter = subtract_overlap(gray_matter, amygdala)

# thalamus
thalamus = subtract_overlap(thalamus, hippocampus)
thalamus = subtract_overlap(thalamus, amygdala)

# amygdala
amygdala = subtract_overlap(amygdala, hippocampus)

# Copy information from original images
white_matter.CopyInformation(white_matter_original)
gray_matter.CopyInformation(gray_matter_original)
thalamus.CopyInformation(thalamus_original)
hippocampus.CopyInformation(hippocampus_original)
amygdala.CopyInformation(amygdala_original)

# Save the images
sitk.WriteImage(white_matter, "overlapping_output/117122/white_matter_r.mha")
sitk.WriteImage(gray_matter, "overlapping_output/117122/gray_matter_r.mha")
sitk.WriteImage(thalamus, "overlapping_output/117122/thalamus_r.mha")
sitk.WriteImage(hippocampus, "overlapping_output/117122/hippocampus_r.mha")
sitk.WriteImage(amygdala, "overlapping_output/117122/amygdala_r.mha")

C:\Users\FlipFlop\Documents\UniBE\Sem5_MedImLab\Repo\MIALab
Current Working Directory: C:\Users\FlipFlop\Documents\UniBE\Sem5_MedImLab\Repo\MIALab


In [12]:
def calculate_overlap_matrix(segmentation_images):
    """Calculates the overlap matrix for the given segmentation images."""

    # Initialize the overlap matrix
    num_labels = len(segmentation_images)
    overlap_matrix = [[0] * num_labels for _ in range(num_labels)]

    # Iterate through all combinations of segmentation images
    for (i, segmentation_image_i), (j, segmentation_image_j) in itertools.product(
        enumerate(segmentation_images), repeat=2
    ):
        # Calculate the intersection (overlap) instead of voxel count difference
        intersection_count = compute_intersection(
            segmentation_image_i, segmentation_image_j
        )
        overlap_matrix[i][j] = intersection_count
        overlap_matrix[j][i] = intersection_count  # Symmetric matrix

    return overlap_matrix



In [13]:
def compute_intersection(structure_a, structure_b):
    """Computes the intersection of two structures."""

    # Set the same origin, spacing, and direction
    structure_a.SetOrigin(structure_b.GetOrigin())
    structure_a.SetSpacing(structure_b.GetSpacing())
    structure_a.SetDirection(structure_b.GetDirection())

    # Compute the intersection
    intersection = sitk.And(structure_a, structure_b)

    # Count non-zero voxels in the intersection
    voxel_count_intersection = sitk.GetArrayFromImage(intersection).sum()

    return voxel_count_intersection


In [14]:
# Read segmentation images
white_matter = sitk.ReadImage(white_matter_path)
gray_matter = sitk.ReadImage(gray_matter_path)
thalamus = sitk.ReadImage(thalamus_path)
hippocampus = sitk.ReadImage(hippocampus_path)
amygdala = sitk.ReadImage(amygdala_path)
segmentation_images= [white_matter, gray_matter, thalamus, hippocampus, amygdala]
overlap_matrix = calculate_overlap_matrix(segmentation_images)
print("Overlap Matrix - original:")
for row in overlap_matrix:
    print(row)

Overlap Matrix - original:
[551804, 0, 15853, 632, 0]
[0, 1002480, 0, 13036, 0]
[15853, 0, 128455, 495, 0]
[632, 13036, 495, 61908, 0]
[0, 0, 0, 0, 27120]


In [15]:
white_matter_path= "overlapping_output/117122/white_matter_r.mha"
gray_matter_path="overlapping_output/117122/gray_matter_r.mha"
thalamus= "overlapping_output/117122/thalamus_r.mha"
hippocampus="overlapping_output/117122/hippocampus_r.mha"
amygdala="overlapping_output/117122/amygdala_r.mha"

# Read segmentation images
white_matter = sitk.ReadImage(white_matter_path)
gray_matter = sitk.ReadImage(gray_matter_path)
thalamus = sitk.ReadImage(thalamus_path)
hippocampus = sitk.ReadImage(hippocampus_path)
amygdala = sitk.ReadImage(amygdala_path)
segmentation_images= [white_matter, gray_matter, thalamus, hippocampus, amygdala]
overlap_matrix = calculate_overlap_matrix(segmentation_images)
print("Overlap Matrix - after substraction:")
for row in overlap_matrix:
    print(row)

Overlap Matrix - original:
[534254, 0, 0, 0, 0]
[0, 985754, 0, 0, 0]
[0, 0, 128455, 495, 0]
[0, 0, 495, 61908, 0]
[0, 0, 0, 0, 27120]


In [25]:
def calculate_weighted_dice_scores(evaluator_results, images_prediction, runinfofile):
    # get dice scores for all labels
    label_mapping = {
        0: 'Background',
        1: 'WhiteMatter',
        2: 'GreyMatter',
        3: 'Hippocampus',
        4: 'Amygdala',
        5: 'Thalamus'
    }
    def get_dice_scores_for_all_labels(results):
        unique_labels = set(
            result.label for result in results if result.metric == 'DICE')
        dice_scores_per_label = {}
        for label in unique_labels:
            dice_scores_per_label[label] = [result.value for result in results if
                                            result.metric == 'DICE' and result.label == label]
        return dice_scores_per_label


    def get_voxel_count_per_label(img):
        image_array = sitk.GetArrayFromImage(img)
        unique_labels = np.unique(image_array)
        # volume_per_label = []
        volume_per_label = {}
        for label in unique_labels:
            # Extract the voxel indices for the current label
            label_indices = np.where(image_array == label)

            # Calculate the size of the label in each dimension
            size_x = (np.max(label_indices[0]) - np.min(label_indices[0]) + 1)
            size_y = (np.max(label_indices[1]) - np.min(label_indices[1]) + 1)
            size_z = (np.max(label_indices[2]) - np.min(label_indices[2]) + 1)

            volume = size_x * size_y * size_z
            volume_per_label[label] = volume
        return volume_per_label


    def map_fields(init_dict, map_dict, res_dict=None):
        res_dict = res_dict or {}
        for k, v in init_dict.items():
            if isinstance(v, dict):
                v = map_fields(v, map_dict[k])
            elif k in map_dict.keys():
                k = str(map_dict[k])
            res_dict[k] = v
        return res_dict


    def calculate_weighted_dice_score_per_label(dice_scores_per_label, volume_per_label):

        background_label=volume_per_label["Background"]
        del volume_per_label["Background"]  # remove background
        if len(dice_scores_per_label) != len(volume_per_label):
            volume_per_label = {label: volume_per_label[label] for label in dice_scores_per_label}

        # Calculate the total voxel count
        #total_voxel_count = sum(volume_per_label.values())
        total_voxel_count = background_label

        voxel_counts_ordered = {label: volume_per_label[label] for label in dice_scores_per_label}

        weighted_dice_scores = {
            label: (total_voxel_count - voxel_counts_ordered[label]) * dice_scores_per_label[label] / total_voxel_count
            for label in dice_scores_per_label if dice_scores_per_label[label] != 0.0
        }


        return weighted_dice_scores


    dice_scores_all_labels = get_dice_scores_for_all_labels(evaluator_results)

    # Remove the second entry [1] from each list (for some reason I get double entries for each label)
    dice_scores_all_labels = {label: score[0] for label, score in dice_scores_all_labels.items()}

    #Get voxel count per label
    volume_per_img = [map_fields(get_voxel_count_per_label(img),label_mapping) for img in images_prediction]

    # Calculate the total voxel count per label
    voxel_count_per_label = {}
    for volume_dict in volume_per_img:
        for label, volume in volume_dict.items():
            if label not in voxel_count_per_label:
                voxel_count_per_label[label] = []
            voxel_count_per_label[label].append(volume)

    # Calculate the mean voxel count per label
    #fixme this one might be pointless
    mean_voxel_count_per_label = {label: np.mean(volumes) for label, volumes in voxel_count_per_label.items()}
    # fixme background same as total_voxels_avg???
    #del mean_voxel_count_per_label["Background"]#remove background

    weighted_dice_scores = calculate_weighted_dice_score_per_label(dice_scores_all_labels, mean_voxel_count_per_label)

    print("Weighted Dice Scores per Label:")
    for label, score in weighted_dice_scores.items():
        print(f"Label {label}: {score}")
    with open(runinfofile, 'a') as f:
        f.write("\nWeighted Dice Scores:\n")
        for label, score in weighted_dice_scores.items():
            f.write(f"Label {label}: {score}\n")

In [28]:
import mialab.utilities.pipeline_utilities as putil
    
# load atlas images
image_prediction = sitk.ReadImage(white_matter_path)
script_dir = r"C:\Users\FlipFlop\Documents\UniBE\Sem5_MedImLab\Repo\MIALab\bin"
data_atlas_dir=os.path.normpath(os.path.join(script_dir, '../data/atlas'))
putil.load_atlas_images(data_atlas_dir)

LOADING_KEYS = [structure.BrainImageTypes.T1w,
                structure.BrainImageTypes.T2w,
                structure.BrainImageTypes.GroundTruth,
                structure.BrainImageTypes.BrainMask,
                structure.BrainImageTypes.RegistrationTransform]

import mialab.utilities.file_access_utilities as futil
# crawl the training image directories
data_test_dir=os.path.normpath(os.path.join(script_dir, '../data/test/'))
crawler = futil.FileSystemDataCrawler(data_test_dir,
                                      LOADING_KEYS,
                                      futil.BrainImageFilePathGenerator(),
                                      futil.DataDirectoryFilter())
pre_process_params = {'skullstrip_pre': True,
                      'normalization_pre': True,
                      'registration_pre': True,
                      'coordinates_feature': True,
                      'intensity_feature': True,
                      'gradient_intensity_feature': True}
binary_label = 1 #todo white matter
# Load images for training and preprocess
images_test = putil.pre_process_batch(crawler.data, pre_process_params, multi_process=False, label=binary_label)

binary_label=1
for img in images_test:
    evaluator = putil.init_evaluator(binary_label)#todo white matter
    evaluator.evaluate(image_prediction, img.images[structure.BrainImageTypes.GroundTruth], img.id_)

result_dir=os.path.normpath(os.path.join(script_dir, '../overlap-result'))
print('\nSubject-wise results...')
result_file = os.path.join(result_dir, f'results_{binary_label}.csv')

import pymia.evaluation.writer as writer


    
print('\nSubject-wise results...')
writer.ConsoleWriter(use_logging=True).write(evaluator.results)

# report also mean and standard deviation among all subjects
result_summary_file = os.path.join(result_dir, 'results_summary.csv')
functions = {'MEAN': np.mean, 'STD': np.std}
writer.CSVStatisticsWriter(result_summary_file, functions=functions).write(evaluator.results)
print('\nAggregated statistic results...')
writer.ConsoleStatisticsWriter(functions=functions).write(evaluator.results)

WEIGHTED_DICE=True
images_prediction=[]
images_prediction.append(image_prediction)

# # write run info file
runinfofile = os.path.join(result_dir, 'RunInfo.txt')
with open(runinfofile, 'w') as f:
    f.write("General info:\n")
    f.write("label: " + str(binary_label) + "\n")


if WEIGHTED_DICE:
    calculate_weighted_dice_scores(evaluator.results, images_prediction, runinfofile)

# clear results such that the evaluator is ready for the next evaluation
evaluator.clear()




---------- Processing 117122
---------- Processing 118528
---------- Processing 118730
---------- Processing 118932
---------- Processing 120111
---------- Processing 122317
---------- Processing 122620
---------- Processing 123117
---------- Processing 123925
---------- Processing 124422

Subject-wise results...

Subject-wise results...

Aggregated statistic results...
LABEL        METRIC   STATISTIC  VALUE
WhiteMatter  DICE     MEAN       0.544
WhiteMatter  DICE     STD        0.000
WhiteMatter  HDRFDST  MEAN       6.481
WhiteMatter  HDRFDST  STD        0.000
Weighted Dice Scores per Label:
Label WhiteMatter: 0.3189080987254304
